# Kili Tutorial: Importing assets

In this tutorial, we will walk through the process of using Kili to import assets. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way.

Additionally:

For an overview of Kili, visit kili-technology.com
You can also check out the Kili documentation https://kili-technology.github.io/kili-docs.

The tutorial is divided into four parts:

1. Importing assets
2. Setting up a labeling priority
3. Setting up who can label the asset

This next cell connect the notebook to the Kili API. You need to update the credentials `email`, `password` and `project_id` before.

In [ ]:
!pip install kili
from kili.authentication import KiliAuth
from kili.playground import Playground

email = 'YOUR EMAIL'
password = 'YOUR PASSWORD'
project_id = 'YOUR PROJECT ID'

kauth = KiliAuth(email=email, password=password)
playground = Playground(kauth)

## Importing assets

An Asset is the smallest piece of information we want to label. It can be: a text, an image, a sound, etc...

To insert an asset in Kili, you need two essential pieces of information:

- The content of the asset (i.e. the text for a text document, or an url that points to the document, or the path to the image file on the hard drive).
- The name of the asset in Kili to be able to recognize it once inserted: `externalId`. This name must be unique within the same project.

It is also possible to associate meta information to certain assets that can be displayed in certain interfaces. We will not go into these details in this tutorial.

The cell below inserts three image files: two are in the form of image urls and one is in the form of a file on the hard drive. **Warning, before running this snippet, make sure you have an `example.jpg` file in your Downloads folder.**

In [ ]:
from tqdm import tqdm

CHUNK_SIZE = 100

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


assets = [
    {
        'externalId': 'example 1',
        'content': 'https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg',
        'metadata': {},
        'toBeLabeledBy':['jean.dupont@example.com']
    },
    {
        'externalId': 'example 2',
        'content': 'https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg',
        'metadata': {},
        'priority': 1
    },
    {
        'externalId': 'example 3',
        'content': os.path.join(os.getenv('HOME'), 'Downloads', 'example.jpg'),
        'metadata': {},
    },
]


for asset_chunk in tqdm(list(chunks(assets, CHUNK_SIZE))):
    external_id_array = [a.get('externalId') for a in asset_chunk]
    content_array = [a.get('content') for a in asset_chunk]
    json_metadata_array = [json.loads(a.get('metadata')) for a in asset_chunk]
    playground.append_many_to_dataset(project_id=project_id, 
                                      content_array=content_array,
                                      external_id_array=external_id_array, 
                                      json_metadata_array=json_metadata_array)

## Setting up a labeling priority
In Kili, by default, the order in which the assets are labeled is the order in which they are inserted in Kili.

It is possible to change this order at any time, for example for active learning. To put an asset at the beginning of the stack, just change its `priority` property, which by default is 0, to a higher value. The higher the `priority` property, the higher the priority of the asset.

The cell below will put the asset `example 2` at the top of the stack.

In [ ]:
for asset in assets:
    if 'priority' not in asset:
        continue
    playground.update_properties_in_asset(asset_id=asset.get('externalId'), 
                                          priority=asset.get('priority'))

## Setting up who can label the asset
In some cases we also want to be able to choose who among the annotators will label the assets. To do this, simply pass to the asset the list of emails from the annotators in question with the `toBeLabeledBy` property.

The cell below will allow asset `example 1` to be only seen by jean.dupont@example.com.

In [ ]:
for asset in assets:
    if 'toBeLabeledBy' not in asset:
        continue
    playground.update_properties_in_asset(asset_id=asset.get('externalId'), 
                                          to_be_labeled_by=asset.get('toBeLabeledBy'))

In this tutorial, we accomplished the following:

We introduced the concept of Kili asset content and asset external ID. We demonstrated how to append an asset to a project with the API.
We showed that an an asset can have a priority and a list a labelers.
If you enjoyed this tutorial, check out the other Recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the [Kili website](https://kili-technology.com) or [Kili documentation](https://kili-technology.github.io/kili-docs) for more info!